<span style="color:#ECEFF4">

# Entrega 3 - Conclusiones
# "Informalidad laboral en Chile: un análisis en base a la Encuesta Nacional de Empleo"
# Nombre: Nicolás Torres Hormazábal


<span style="color:#ECEFF4">

### Este proyecto tiene como propósito desarrollar un modelo de ciencia de datos que explore y analice las características sociodemográficas, geográficas, educativas y otras variables que influyen en la probabilidad de que una persona opte o se vea obligada a trabajar en el sector informal. El objetivo es explicar el fenómeno de la informalidad y entender mejor a quienes forman parte de este sector, utilizando los resultados de la encuesta.
### En esta entrega final, se presentará un modelo de redes neuronales con su respectiva optimización de hiperparametros usando optuna, así como correcciones respecto de la entrega pasada.
### Finalmente, se presentarán las conclusiones y propuestas de mejora para el proyecto realizado, también para el diseño de la encuesta.

In [2]:
# Tema y colores para nuestros gráficos
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import logging

# Configurar logging para suprimir mensajes de advertencia de matplotlib
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

# Ruta al archivo de la fuente personalizada
#CAMBIAR ESTO PARA QUE FUNCIONE CON VSC
font_path = r'D:\Google Drive\Mi unidad\TAAA\Entrega_1\FiraSans-Medium.ttf'  # Asegúrate de que esta ruta es correcta
#font_path = r'/content/drive/MyDrive/TAAA/Entrega_1/FiraSans-Medium.ttf'
prop = fm.FontProperties(fname=font_path)  # Cargar la fuente desde el archivo

# Registrar la fuente personalizada en matplotlib
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = prop.get_name()  # Usar Fira Sans como fuente predeterminada

# Colores del tema Nord Aurora
nord_aurora_colors = {
    'background': '#ECEFF4',   # Fondo claro
    'grid': '#4C566A',         # Color de la cuadrícula
    'text': '#2e3440',         # Texto oscuro
    'accent1': '#C3202F',      # Rojo intenso
    'accent2': '#EBCB8B',      # Amarillo Aurora
    'accent3': '#A3BE8C',      # Verde Aurora
    'accent4': '#B48EAD',      # Morado Aurora
}

# Configurar matplotlib con el tema Nord Aurora y Fira Sans como predeterminada
plt.rcParams.update({
    'figure.facecolor': nord_aurora_colors['background'],
    'axes.facecolor': nord_aurora_colors['background'],
    'axes.edgecolor': nord_aurora_colors['grid'],
    'axes.labelcolor': nord_aurora_colors['text'],
    'axes.titlecolor': nord_aurora_colors['text'],
    'xtick.color': nord_aurora_colors['text'],
    'ytick.color': nord_aurora_colors['text'],
    'grid.color': nord_aurora_colors['grid'],
    'grid.linewidth': 0,  # Desactivar la cuadrícula
    'legend.frameon': False,
    'legend.fontsize': 12,
    'legend.labelcolor': nord_aurora_colors['text'],
    'text.color': nord_aurora_colors['text'],
    'axes.spines.top': False,   # Ocultar línea superior del eje
    'axes.spines.right': False, # Ocultar línea derecha del eje
    'axes.prop_cycle': plt.cycler(color=[
        nord_aurora_colors['accent1'],
        nord_aurora_colors['accent2'],
        nord_aurora_colors['accent3'],
        nord_aurora_colors['accent4']
    ]),
    'figure.figsize': (18, 9),
    'axes.titlesize': 16,
    'axes.titleweight': 'bold',
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
})

# Configurar seaborn con el mismo estilo de fondo y colores Nord Aurora, y ocultar spines superior y derecha
sns.set_theme(
    style="whitegrid",  # Utilizar cuadrícula blanca por defecto
    rc={
        "axes.facecolor": nord_aurora_colors['background'],
        "figure.facecolor": nord_aurora_colors['background'],
        "axes.edgecolor": nord_aurora_colors['grid'],
        "xtick.color": nord_aurora_colors['text'],
        "ytick.color": nord_aurora_colors['text'],
        "axes.labelcolor": nord_aurora_colors['text'],
        "axes.titlecolor": nord_aurora_colors['text'],
        "axes.spines.top": False,    # Ocultar línea superior del eje
        "axes.spines.right": False,  # Ocultar línea derecha del eje
        "font.family": prop.get_name(),   # Establecer Fira Sans como fuente predeterminada
    }
)

# Definir una paleta personalizada para seaborn usando los acentos de Nord Aurora
nord_palette = [nord_aurora_colors['accent1'], nord_aurora_colors['accent2'], nord_aurora_colors['accent3'], nord_aurora_colors['accent4']]
sns.set_palette(nord_palette)

In [3]:
# Colores para ocupados
palette = {
    'Ocupado Formal': '#263252',  # Azul intenso para Ocupado Formal
    'Ocupado Informal': '#fc4b08' # Naranja
}

In [4]:
# para que muestre los gráficos
%matplotlib inline

In [5]:
#ILibrerías
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#ignorar todos los warnings (esto lo hicimos una vez ya realizamos todo el procesamiento y graficos)
#es solo por estética
#warnings.filterwarnings("ignore")
#con esto mostramos todas las columnas de los dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [10]:
df_imputed = pd.read_csv(r'D:\Google Drive\Mi unidad\TAAA\Entrega_3\df_imputed.csv', sep=',', encoding='latin-1')

In [11]:
df_imputed.head()

,region,mes_encuesta,r_p_c,tipo,sexo,cine,proveedor,edad,nacionalidad,estudia_actual,orig1,mig1,a1,a2,a3,b1,b2,b5,b7a_1,b7a_2,b7a_3,b7b_2,b7b_3,b8,b12,b11_proxy,b14_rev4cl_caenes,i1,i2,i3,ocup_honorarios,plataformas_digitales,turno,habituales,ocup_form,r_p_rev4cl_caenes
0,16,1,16203,1,2,6,0,24,152,2,2,1,1.0,99.0,1.0,5.0,2.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,99.0,1.0,20.0,99.0,99.0,99.0,2.0,2.0,2.0,0.343912,0.0,20.0
1,5,1,5101,1,1,8,1,58,152,2,2,1,1.0,99.0,1.0,2.0,1.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,2.0,13.0,99.0,99.0,99.0,2.0,2.0,2.0,0.121710,0.0,13.0
2,3,1,3201,1,1,4,1,61,152,2,2,1,1.0,99.0,1.0,7.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,2.0,-0.767100,1.0,3.0
3,14,1,14201,3,2,5,0,49,152,2,2,1,1.0,99.0,1.0,5.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,16.0,1.0,1.0,1.0,2.0,2.0,2.0,0.343912,0.0,9.0
4,9,1,9102,1,2,7,0,49,152,2,2,1,1.0,99.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,16.0,1.0,1.0,1.0,2.0,2.0,2.0,0.269845,0.0,16.0


In [12]:
from sklearn.model_selection import train_test_split

# Split the data
X = df_imputed.drop(columns=['ocup_form'])
y = df_imputed['ocup_form']
# Entrenamiento = 70%
# Prueba = 30%
# Estratificado en y, ya que está desbalanceado
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [22]:
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152105 entries, 0 to 152104
Data columns (total 36 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   region                 152105 non-null  int64  
 1   mes_encuesta           152105 non-null  int64  
 2   r_p_c                  152105 non-null  int64  
 3   tipo                   152105 non-null  int64  
 4   sexo                   152105 non-null  int64  
 5   cine                   152105 non-null  int64  
 6   proveedor              152105 non-null  int64  
 7   edad                   152105 non-null  int64  
 8   nacionalidad           152105 non-null  int64  
 9   estudia_actual         152105 non-null  int64  
 10  orig1                  152105 non-null  int64  
 11  mig1                   152105 non-null  int64  
 12  a1                     152105 non-null  float64
 13  a2                     152105 non-null  float64
 14  a3                     152105 non-nu

In [25]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from scipy.sparse import hstack

# Separate the column that shouldn't be encoded
X_non_categorical = df_imputed[['habituales']]

# Select all columns except 'habituales' for one-hot encoding
categorical_columns = [col for col in df_imputed.columns if col != 'habituales']
X_categorical = df_imputed[categorical_columns]

# Apply one-hot encoding with sparse matrix using the corrected argument
one_hot_encoder = OneHotEncoder(sparse_output=True, drop='first')  # Corrected 'sparse' to 'sparse_output'
X_categorical_encoded_sparse = one_hot_encoder.fit_transform(X_categorical)

# Convert the 'habituales' column to a sparse matrix for consistency
X_non_categorical_sparse = X_non_categorical.to_numpy()

# Combine the sparse matrices
X_final_sparse = hstack([X_non_categorical_sparse, X_categorical_encoded_sparse])


In [40]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, recall_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scipy.sparse import csr_matrix

# Convert sparse matrix to dense if necessary
X_encoded = X_final_sparse  # From previous encoding step
y = df_imputed['ocup_form']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42, stratify=y)

# Normalize the data (sparse compatibility)
scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the objective function for Optuna optimization
def objective(trial):
    # Hyperparameter search space
    num_units = trial.suggest_int('num_units', 4, 64, step=4)  # Number of neurons
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)  # Dropout rate
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])  # Batch size
    epochs = trial.suggest_int('epochs', 5, 50)  # Number of epochs

    # Define the model with the current trial's hyperparameters
    model = Sequential([
        Dense(num_units, input_dim=X_train.shape[1], activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Predict and evaluate using F1 Score and Recall
    y_pred_prob = model.predict(X_test).flatten()
    y_pred = (y_pred_prob > 0.5).astype(int)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Return a metric to minimize (negative F1 Score to maximize it)
    return -f1

# Create the study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Display the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# Train the final model with the best hyperparameters
best_model = Sequential([
    Dense(best_params['num_units'], input_dim=X_train.shape[1], activation='relu'),
    Dropout(best_params['dropout_rate']),
    Dense(1, activation='sigmoid')
])

best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

# Train the model with the best hyperparameters
best_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)

# Evaluate the final model
y_pred_prob = best_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

final_f1 = f1_score(y_test, y_pred)
final_recall = recall_score(y_test, y_pred)

# Print final metrics
print(f"\nFinal Test F1 Score: {final_f1}")
print(f"Final Test Recall: {final_recall}")
print("\nFinal Classification Report:")
print(classification_report(y_test, y_pred))


[I 2025-01-07 13:09:48,120] A new study created in memory with name: no-name-6ad5f5c7-65e4-42ca-9118-5ec5537e3558
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1426/1426 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


[I 2025-01-07 13:16:21,463] Trial 0 finished with value: -0.9999619178186526 and parameters: {'num_units': 40, 'learning_rate': 0.0015503922405725684, 'dropout_rate': 0.4352430071755233, 'batch_size': 16, 'epochs': 44}. Best is trial 0 with value: -0.9999619178186526.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1426/1426 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


[I 2025-01-07 13:18:31,090] Trial 1 finished with value: -0.9999238327366897 and parameters: {'num_units': 48, 'learning_rate': 0.004578004638964526, 'dropout_rate': 0.05645178572387638, 'batch_size': 64, 'epochs': 49}. Best is trial 0 with value: -0.9999619178186526.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

1426/1426 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


[I 2025-01-07 13:56:54,249] Trial 18 finished with value: -0.999885753455958 and parameters: {'num_units': 4, 'learning_rate': 0.00011734227121926448, 'dropout_rate': 0.26809988176070265, 'batch_size': 32, 'epochs': 31}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1426/1426 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


[I 2025-01-07 14:04:10,475] Trial 19 finished with value: -0.9999619178186526 and parameters: {'num_units': 64, 'learning_rate': 0.006942466148802547, 'dropout_rate': 0.20888328356208974, 'batch_size': 16, 'epochs': 40}. Best is trial 2 with value: -1.0.
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best Hyperparameters: {'num_units': 28, 'learning_rate': 0.006358720879026974, 'dropout_rate': 0.4908199591525962, 'batch_size': 32, 'epochs': 21}
Epoch 1/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9692 - loss: 0.0823
Epoch 2/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9975 - loss: 0.0099
Epoch 3/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9986 - loss: 0.0054
Epoch 4/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9990 - loss: 0.0050
Epoch 5/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9994 - loss: 0.0027
Epoch 6/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9995 - loss: 0.0031
Epoch 7/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9996 - loss: 0.0026
Epoch 8/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9995 - loss: 0.0035
Epoch 9/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9997 - loss: 0.0016
Epoch 10/21
3328/3328 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accu

# Optuna se demoró 56 minutos y obtuvo

# Best Hyperparameters: {'num_units': 28, 'learning_rate': 0.006358720879026974, 'dropout_rate': 0.4908199591525962, 'batch_size': 32, 'epochs': 21}

In [ ]:
[I 2025-01-07 13:09:48,120] A new study created in memory with name: no-name-6ad5f5c7-65e4-42ca-9118-5ec5537e3558
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:16:21,463] Trial 0 finished with value: -0.9999619178186526 and parameters: {'num_units': 40, 'learning_rate': 0.0015503922405725684, 'dropout_rate': 0.4352430071755233, 'batch_size': 16, 'epochs': 44}. Best is trial 0 with value: -0.9999619178186526.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:18:31,090] Trial 1 finished with value: -0.9999238327366897 and parameters: {'num_units': 48, 'learning_rate': 0.004578004638964526, 'dropout_rate': 0.05645178572387638, 'batch_size': 64, 'epochs': 49}. Best is trial 0 with value: -0.9999619178186526.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:20:12,561] Trial 2 finished with value: -1.0 and parameters: {'num_units': 28, 'learning_rate': 0.006358720879026974, 'dropout_rate': 0.4908199591525962, 'batch_size': 32, 'epochs': 21}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:23:50,801] Trial 3 finished with value: -1.0 and parameters: {'num_units': 12, 'learning_rate': 0.0018222307287853704, 'dropout_rate': 0.42713620552241366, 'batch_size': 32, 'epochs': 49}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:26:53,148] Trial 4 finished with value: -0.9999619178186526 and parameters: {'num_units': 20, 'learning_rate': 0.001238577762828095, 'dropout_rate': 0.2848202368198418, 'batch_size': 16, 'epochs': 21}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:29:10,079] Trial 5 finished with value: -0.9999238385377 and parameters: {'num_units': 60, 'learning_rate': 0.000326783659760162, 'dropout_rate': 0.17243097337774088, 'batch_size': 64, 'epochs': 46}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:35:49,143] Trial 6 finished with value: -0.9999238385377 and parameters: {'num_units': 20, 'learning_rate': 0.0002122098450269737, 'dropout_rate': 0.45639561253918853, 'batch_size': 16, 'epochs': 45}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:37:03,377] Trial 7 finished with value: -0.9999619178186526 and parameters: {'num_units': 20, 'learning_rate': 0.0003083441503502718, 'dropout_rate': 0.43978290592233005, 'batch_size': 64, 'epochs': 28}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:38:10,526] Trial 8 finished with value: -0.9999619178186526 and parameters: {'num_units': 8, 'learning_rate': 0.00020351176771513774, 'dropout_rate': 0.4471957942695193, 'batch_size': 64, 'epochs': 28}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:38:50,042] Trial 9 finished with value: -0.9998095600837935 and parameters: {'num_units': 12, 'learning_rate': 0.0023424770244325818, 'dropout_rate': 0.20555052862410172, 'batch_size': 32, 'epochs': 8}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:40:02,076] Trial 10 finished with value: -0.9999619178186526 and parameters: {'num_units': 36, 'learning_rate': 0.009327824383419565, 'dropout_rate': 0.31826741660761526, 'batch_size': 32, 'epochs': 14}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:42:56,403] Trial 11 finished with value: -0.9999619178186526 and parameters: {'num_units': 28, 'learning_rate': 0.00375003178457329, 'dropout_rate': 0.3550165858644073, 'batch_size': 32, 'epochs': 37}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:45:45,231] Trial 12 finished with value: -0.9999238327366897 and parameters: {'num_units': 4, 'learning_rate': 0.0006190866096189092, 'dropout_rate': 0.4870690201510216, 'batch_size': 32, 'epochs': 35}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:47:17,738] Trial 13 finished with value: -1.0 and parameters: {'num_units': 28, 'learning_rate': 0.009708298108800853, 'dropout_rate': 0.36613474306918403, 'batch_size': 32, 'epochs': 19}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:50:14,128] Trial 14 finished with value: -0.9999619178186526 and parameters: {'num_units': 48, 'learning_rate': 0.004535511026727191, 'dropout_rate': 0.37569335506398027, 'batch_size': 32, 'epochs': 35}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:52:08,726] Trial 15 finished with value: -0.9999619178186526 and parameters: {'num_units': 12, 'learning_rate': 0.0005918663379075565, 'dropout_rate': 0.11373001917328729, 'batch_size': 32, 'epochs': 21}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:52:46,342] Trial 16 finished with value: -0.9999619178186526 and parameters: {'num_units': 28, 'learning_rate': 0.0023371868182299493, 'dropout_rate': 0.4944407035416798, 'batch_size': 32, 'epochs': 6}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 1ms/step
[I 2025-01-07 13:54:02,897] Trial 17 finished with value: -0.9999619178186526 and parameters: {'num_units': 16, 'learning_rate': 0.0007580820079815748, 'dropout_rate': 0.3915487529623783, 'batch_size': 32, 'epochs': 14}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m3s[0m 2ms/step
[I 2025-01-07 13:56:54,249] Trial 18 finished with value: -0.999885753455958 and parameters: {'num_units': 4, 'learning_rate': 0.00011734227121926448, 'dropout_rate': 0.26809988176070265, 'batch_size': 32, 'epochs': 31}. Best is trial 2 with value: -1.0.
C:\Users\nicot\AppData\Local\Temp\ipykernel_3208\2417410773.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)  # Learning rate
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 2ms/step
[I 2025-01-07 14:04:10,475] Trial 19 finished with value: -0.9999619178186526 and parameters: {'num_units': 64, 'learning_rate': 0.006942466148802547, 'dropout_rate': 0.20888328356208974, 'batch_size': 16, 'epochs': 40}. Best is trial 2 with value: -1.0.
c:\Users\nicot\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Best Hyperparameters: {'num_units': 28, 'learning_rate': 0.006358720879026974, 'dropout_rate': 0.4908199591525962, 'batch_size': 32, 'epochs': 21}
Epoch 1/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m8s[0m 2ms/step - accuracy: 0.9692 - loss: 0.0823
Epoch 2/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9975 - loss: 0.0099
Epoch 3/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9986 - loss: 0.0054
Epoch 4/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9990 - loss: 0.0050
Epoch 5/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9994 - loss: 0.0027
Epoch 6/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9995 - loss: 0.0031
Epoch 7/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9996 - loss: 0.0026
Epoch 8/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9995 - loss: 0.0035
Epoch 9/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9997 - loss: 0.0016
Epoch 10/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9995 - loss: 0.0034
Epoch 11/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m7s[0m 2ms/step - accuracy: 0.9998 - loss: 0.0011
Epoch 12/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m7s[0m 2ms/step - accuracy: 0.9997 - loss: 0.0022
Epoch 13/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m7s[0m 2ms/step - accuracy: 0.9999 - loss: 6.1396e-04
Epoch 14/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m7s[0m 2ms/step - accuracy: 0.9996 - loss: 0.0019
Epoch 15/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m7s[0m 2ms/step - accuracy: 0.9998 - loss: 0.0019
Epoch 16/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m7s[0m 2ms/step - accuracy: 0.9999 - loss: 8.5804e-04
Epoch 17/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9998 - loss: 0.0014
Epoch 18/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9999 - loss: 8.1631e-04
Epoch 19/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9998 - loss: 0.0010
Epoch 20/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m6s[0m 2ms/step - accuracy: 0.9998 - loss: 0.0014
Epoch 21/21
[1m3328/3328[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m7s[0m 2ms/step - accuracy: 0.9999 - loss: 8.1362e-04
[1m1426/1426[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 2ms/step

Final Test F1 Score: 1.0
Final Test Recall: 1.0

Final Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     32503
         1.0       1.00      1.00      1.00     13129

    accuracy                           1.00     45632
   macro avg       1.00      1.00      1.00     45632
weighted avg       1.00      1.00      1.00     45632

In [41]:
# Print final metrics
print(f"\nFinal Test F1 Score: {final_f1}")
print(f"Final Test Recall: {final_recall}")
print("\nFinal Classification Report:")
print(classification_report(y_test, y_pred))


Final Test F1 Score: 1.0
Final Test Recall: 1.0

Final Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     32503
         1.0       1.00      1.00      1.00     13129

    accuracy                           1.00     45632
   macro avg       1.00      1.00      1.00     45632
weighted avg       1.00      1.00      1.00     45632

